<a href="https://colab.research.google.com/github/forouzanfallah/Convert_PSCAD_Out_to_CSV/blob/main/Convert_Out_to_NPY_Customized.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Connect to Google Drive

In [11]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


###NPY Format

In [2]:
import pandas as pd
import numpy as np

# Step 1: Read CSV Data
df = pd.read_csv('/content/drive/MyDrive/dataCig/Bus23.csv')

# Step 2: Preprocess Data

# Step 3: Convert to NumPy Array
np_array = df.to_numpy()

# Step 4: Save as `.npy` File
np.save('Bus23.npy', np_array)


In [9]:
import pandas as pd
import numpy as np

# Read the .csv file
csv_df = pd.read_csv('/content/drive/MyDrive/dataCig/Bus23.csv')

# Read the .npy file and convert it to a pandas DataFrame
npy_data = np.load('/content/Bus23.npy')
npy_df = pd.DataFrame(npy_data)

# Compare the two DataFrames
# Check if they have the same shape
same_shape = csv_df.shape == npy_df.shape

# Check if they have the same data values, ignoring column names
same_values = np.array_equal(csv_df.values, npy_df.values)

# Output the results
print("Same Shape:", same_shape)
print("Same Values:", same_values)


Same Shape: True
Same Values: True


###Directories

In [12]:
import zipfile
import os

# Replace this with the path to your ZIP file
zip_file_path = '/content/drive/MyDrive/dataCig/data.zip'

# Replace this with the path where you want to extract the files and store CSV files
data_directory = '/content/CIGRE'

os.makedirs(data_directory, exist_ok=True)

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(data_directory)

print(f"Files extracted to {data_directory}")


Files extracted to /content/CIGRE


###OUT to CSV

In [13]:
import pandas as pd
import re
import os

# Define your headers here. This is just an example.
headers = ['time', 'I12:1', 'I12:2', 'I12:3', 'I1213:1', 'I1213:2', 'I1213:3', 'I23:1', 'I23:2', 'I23:3', 'I1314:1', 'I1314:2', 'I1314:3', 'I34:1', 'I34:2', 'I34:3', 'I38:1', 'I38:2', 'I38:3', 'I12:1', 'I12:2', 'I12:3', 'I13:1', 'I13:2', 'I13:3', 'I148:1', 'I148:2', 'I148:3', 'I14:1', 'I14:2', 'I14:3', 'I83:1', 'I83:2', 'I83:3', 'I45:1', 'I45:2', 'I45:3', 'I89:1', 'I89:2', 'I89:3', 'I411:1', 'I411:2', 'I411:3', 'I87:1', 'I87:2', 'I87:3', 'I114:1', 'I114:2', 'I114:3', 'I76:1', 'I76:2', 'I76:3', 'I1110:1', 'I1110:2', 'I1110:3', 'I910:1', 'I910:2', 'I910:3', 'I56:1', 'I56:2', 'I56:3', 'I67:1', 'I67:2', 'I67:3', 'I65:1', 'I65:2', 'I65:3', 'I109:1', 'I109:2', 'I109:3', 'I1011:1', 'I1011:2', 'I1011:3', 'I54:1', 'I54:2', 'I54:3', 'I98:1', 'I98:2', 'I98:3', 'I78:1', 'I78:2', 'I78:3', 'I814:1', 'I814:2', 'I814:3', 'I43:1', 'I43:2', 'I43:3', 'I1413:1', 'I1413:2', 'I1413:3', 'I11:1', 'I11:2', 'I11:3', 'I32:1', 'I32:2', 'I32:3', 'I10:1', 'I10:2', 'I10:3', 'I9:1', 'I9:2', 'I9:3', 'I1312:1', 'I1312:2', 'I1312:3', 'I8:1', 'I8:2', 'I8:3', 'I7:1', 'I7:2', 'I7:3', 'I21:1', 'I21:2', 'I21:3', 'I6:1', 'I6:2', 'I6:3', 'I5:1', 'I5:2', 'I5:3', 'I4:1', 'I4:2', 'I4:3', 'I3:1', 'I3:2', 'I3:3', 'I2:1', 'I2:2', 'I2:3', 'I1:1', 'I1:2', 'I1:3']

for folder in os.listdir(data_directory):
    folder_path = os.path.join(data_directory, folder)

    if os.path.isdir(folder_path):
        # List .out files in the directory
        file_names = [f for f in os.listdir(folder_path) if f.endswith('.out')]
        file_names.sort(key=lambda x: int(re.search(r'_(\d+)', x).group(1)))
        print(file_names)

        dfs = []
        for idx, file_name in enumerate(file_names):
            try:
                with open(os.path.join(folder_path, file_name), 'r') as file:
                    lines = file.readlines()
                data = [[float(val) for val in line.split()] for line in lines]
                df = pd.DataFrame(data)
                if idx > 0:
                    df = df.drop(df.columns[0], axis=1)
                dfs.append(df)
            except Exception as e:
                print(f"Error reading {file_name}: {e}")

        final_df = pd.concat(dfs, axis=1)

        # Adjust headers based on the number of columns in final_df
        num_cols = final_df.shape[1]
        if num_cols > len(headers):
            # Extend the headers list if there are more columns than predefined headers
            final_df = final_df.iloc[:, :len(headers)]
        elif num_cols < len(headers):
            # Truncate the headers list if there are fewer columns than predefined headers
            headers = headers[:num_cols]

        final_df.columns = headers

        # # Save the DataFrame to a CSV file
        # final_df.to_csv(os.path.join(data_directory, f'{folder}.csv'), index=False)
        # print(f'CSV file saved for {folder}')

        # Save the NumPy array to a .npy file
        data_array = final_df.to_numpy()
        npy_file_path = os.path.join(data_directory, f'{folder}.npy')
        np.save(npy_file_path, data_array)
        print(f'Numpy .npy file saved for {folder}')


['Bus23_0.out', 'Bus23_1.out', 'Bus23_2.out', 'Bus23_3.out', 'Bus23_4.out', 'Bus23_5.out', 'Bus23_6.out', 'Bus23_7.out', 'Bus23_8.out', 'Bus23_9.out', 'Bus23_10.out', 'Bus23_11.out', 'Bus23_12.out', 'Bus23_13.out']
Numpy .npy file saved for Bus23
['Bus38_0.out', 'Bus38_1.out', 'Bus38_2.out', 'Bus38_3.out', 'Bus38_4.out', 'Bus38_5.out', 'Bus38_6.out', 'Bus38_7.out', 'Bus38_8.out', 'Bus38_9.out', 'Bus38_10.out', 'Bus38_11.out', 'Bus38_12.out', 'Bus38_13.out']
Numpy .npy file saved for Bus38
['Bus78_0.out', 'Bus78_1.out', 'Bus78_2.out', 'Bus78_3.out', 'Bus78_4.out', 'Bus78_5.out', 'Bus78_6.out', 'Bus78_7.out', 'Bus78_8.out', 'Bus78_9.out', 'Bus78_10.out', 'Bus78_11.out', 'Bus78_12.out', 'Bus78_13.out']
Numpy .npy file saved for Bus78
['Bus34_0.out', 'Bus34_1.out', 'Bus34_2.out', 'Bus34_3.out', 'Bus34_4.out', 'Bus34_5.out', 'Bus34_6.out', 'Bus34_7.out', 'Bus34_8.out', 'Bus34_9.out', 'Bus34_10.out', 'Bus34_11.out', 'Bus34_12.out', 'Bus34_13.out']
Numpy .npy file saved for Bus34
['normal_0.o

###Download CSV files

In [ ]:
import os
from google.colab import files
for filename in os.listdir(data_directory):
    if filename.endswith('.csv'):
        file_path = os.path.join(data_directory, filename)
        # Download the file
        files.download(file_path)


###Download NPY files

In [14]:
import os
from google.colab import files
for filename in os.listdir(data_directory):
    if filename.endswith('.npy'):
        file_path = os.path.join(data_directory, filename)
        # Download the file
        files.download(file_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>